In [37]:
import pandas as pd
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

In [61]:
df = pd.read_excel('./data/iniall.xlsx', index_col=0)
df 

,工作天数,已工作天数,驾驶资格,历史事故,已安排A,已安排B,已安排C,已安排D
编号,,,,,,,,
1,6,4,1,1,2,0,0,0
2,11,6,1,0,1,1,0,0
3,9,8,1,1,1,2,1,0
4,10,5,1,2,1,1,1,0
5,13,8,1,3,2,1,0,1
6,15,11,1,4,1,1,1,1
7,6,4,1,2,1,1,0,0
8,10,5,0,5,1,0,1,0
9,5,3,0,3,0,0,1,1


In [74]:
class IP():
    def __init__(self, df):
        self.df = df.copy()
        self.w = [0.5, 0.2, 0.1, 1]

    def RunMultiDays(self, rundays=None):
        df = self.df.copy()
        if rundays is None:
            rundays = 3

        for i in range(rundays):
            print('Here is day {}'.format(i+1))
            print(df)
            m, xijres = self.RunSingleDay(df)
            # print(m, xijres)
            df.iloc[np.arange(df.shape[0])[df['已工作天数']>=0],4:4+m] += xijres
            df['已工作天数'] += 1
            df.drop(df.index[ df.loc[:,'已工作天数'] == df.loc[:,'工作天数'] ], axis=0, inplace=True)
            print('\n')

        # for i in range()

    def RunSingleDay(self, df=None):
        if df is None:
            df = self.df.copy()
        df = df.loc[df['已工作天数']>=0,:]
        # 整数规划
        n = df.shape[0]
        if 5<=n<=7:
            m = 1
        elif 8<=n<=10:
            m = 2
        elif 11<=n<=12:
            m = 3
        elif 13<=n<=15:
            m = 4
        else:
            raise ValueError('Error: Number of Workers is {}'.format(n))

        xij = cp.Variable((n,m), integer = True)
        tj = cp.Variable(m, integer=True)
        WorkFrac = cp.Variable(n)
        AcciNote = cp.Variable(m, integer = True)

        TotalWorkDays = df.loc[:, '工作天数'].values
        WorkDays = df.loc[:, '已工作天数'].values + 1
        car = df.loc[:,'驾驶资格'].values
        cars = np.repeat(car.reshape(-1,1), m, axis=1)
        Exps = df.iloc[:, 4:4+m].values
        Accident = df.loc[:,'历史事故'].values
        HighAccident = np.zeros(n)
        LowAccident = np.zeros(n)
        HighAccident[Accident >= 3] = 1
        LowAccident[Accident <= 1] = 1
        HighAccident = HighAccident.reshape(1,-1)
        LowAccident = LowAccident.reshape(1,-1)

        # Create Two Constraints
        constrains = []
        # 0-1变量约束 目标3 事故率尽可能对冲
        constrains.append( AcciNote >= 0)
        # constrains.append( AcciNote <= 1)
        constrains.append( cp.matmul(HighAccident, xij) - cp.matmul(LowAccident, xij) <= cp.reshape(AcciNote, (1,m)) )
        # AcciNote 在 >= 0中可以尽量的小 当且仅当能足够对冲的时候才能取到0，只要我们最小化AcciNote就能确保AcciNote倾向于0
        
        constrains.append( xij >= np.ones((n,m)) * 0 )
        constrains.append( xij <= np.ones((n,m)) )

        constrains.append( cp.sum(xij, axis=0) == np.ones(m) * 2 ) # 条件1
        constrains.append( cp.sum(cp.multiply(xij, cars), axis=0) >= np.ones(m) ) # 条件2
        constrains.append( cp.sum(xij, axis=1) <= np.ones(n) ) # 条件3

        constrains.append( tj >= np.ones(m) * 0 ) # 松弛0-1变量tj
        constrains.append( tj <= np.ones(m) )
        constrains.append( cp.sum(cp.multiply(xij, Exps), axis=0) >= tj ) # 目标1


        ei_bar = cp.sum( (Exps + xij) / m, axis=1) # 目标2
        # cp.sum([cp.sum((xij[:,i] - ei_bar)**2) for i in range(4)]) 

        # cp.sum(AcciNote) # 目标3
        # WorkLoad = (1+ WorkDays + cp.sum(xij, axis=1)) / (1 + TotalWorkDays + cp.sum(xij, axis=1)) 
        # constrains.append( (WorkDays + cp.sum(xij, axis=1)) == WorkFrac )  
        constrains.append( WorkDays + cp.sum(xij, axis=1) == cp.multiply(WorkFrac, TotalWorkDays )  )
        # cp.sum( (WorkLoad - cp.sum(WorkLoad) / n)**2 )
        WorkFracMean = cp.sum(WorkFrac) / n

        # object
        obj = cp.Minimize( - self.w[0] * cp.sum(tj) * 10 + self.w[1] * cp.sum([cp.sum((xij[:,i] - ei_bar)**2) for i in range(m)]) + self.w[2] * 10  * cp.sum(AcciNote) + self.w[3] * 100 * cp.sum( (WorkFrac - cp.sum(WorkFrac)/n)**2 ) )
        # solve
        prob = cp.Problem(obj, constrains)
        prob.solve(solver=cp.CPLEX)

        # print
        xijres = np.zeros((n,m))
        xijres[np.abs(xij.value - 1) < 1e-4] = 1
        print('Opt Target =', prob.value)
        print('xij res =')
        print(xijres)
        print('Targ1 : tj =', tj.value, ' —— 1为满足了')
        print('Targ2 : ', np.sum([np.sum((xij[:,i].value - ei_bar.value)**2) for i in range(m)]))
        print('Targ3 : AcciNote = ', AcciNote.value)
        print('Targ4 : ', np.sum( (WorkFrac.value - np.sum(WorkFrac.value) / n ) ** 2))
        return m, xijres

In [75]:
Instance = IP(df)
Instance.RunSingleDay(df)

Opt Target = 23.44398476306648
xij res =
[[0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]]
Targ1 : tj = [1. 1.]  —— 1为满足了
Targ2 :  21.49999996169643
Targ3 : AcciNote =  [-0. -0.]
Targ4 :  0.29143984770727194


(2,
 array([[0., 0.],
        [0., 1.],
        [0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 1.],
        [0., 0.],
        [1., 0.]]))

In [73]:
Instance.RunMultiDays() 

Here is day 1
    工作天数  已工作天数  驾驶资格  历史事故  已安排A  已安排B  已安排C  已安排D
编号                                                 
1      6      4     1     1     2     0     0     0
2     11      6     1     0     1     1     0     0
3      9      8     1     1     1     2     1     0
4     10      5     1     2     1     1     1     0
5     13      8     1     3     2     1     0     1
6     15     11     1     4     1     1     1     1
7      6      4     1     2     1     1     0     0
8     10      5     0     5     1     0     1     0
9      5      3     0     3     0     0     1     1
10    10      1     0     0     0     0     0     0
11     6     -2     1     0     0     0     0     0
12    10     -3     0     0     0     0     0     0
13     4     -5     1     0     0     0     0     0
14     8     -5     0     0     0     0     0     0
15     5     -6     1     0     0     0     0     0
Opt Target = 30.643984757959338
xij res =
[[0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0